In [103]:
import gurobipy as gp
from gurobipy import GRB
import json
import pandas as pd
import numpy as np
from pathlib import Path

In [104]:
"Loading the Solar Production Data"
data_dir = Path("./")
data_dir_DER_production = data_dir / "DER_production.json"
data_dir_DER_production = data_dir_DER_production.resolve()
with open(f"{data_dir_DER_production}", "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Take the first row's hourly_profile_ratio list and make it a numpy array
Solar_CF = np.array(df["hourly_profile_ratio"][0])

In [105]:
print(f"Solar_CF: {Solar_CF}")

Solar_CF: [0.   0.   0.   0.   0.   0.05 0.14 0.21 0.15 0.12 0.21 0.25 0.85 0.75
 0.55 0.43 0.23 0.05 0.25 0.25 0.   0.   0.   0.  ]


In [106]:
"Fullt Flexible Load (FFL) Data"
# Load JSON filen
data_dir_usage_preferences = data_dir / "usage_preferences.json"
data_dir_usage_preferences = data_dir_usage_preferences.resolve()

with open(f"{data_dir_usage_preferences}", "r") as f:
    data = json.load(f)

    data
FFL_ref = data[0]["load_preferences"][0]["hourly_profile_ratio"]

In [107]:
print(f"FFL_ref: {FFL_ref}")


FFL_ref: [0.055, 0.04, 0.04, 0.04, 0.075, 0.48, 0.76, 0.8, 0.63, 0.22, 0.25, 0.35, 0.3, 0.28, 0.45, 0.65, 0.78, 0.9, 0.98, 0.88, 0.075, 0.15, 0.075, 0.055]


In [108]:
"Electrcity Markert Data"
data_dir_bus_params = data_dir / "bus_params.json"
data_dir_bus_params = data_dir_bus_params.resolve()

with open(f"{data_dir_bus_params}", "r") as f:
    data = json.load(f)

    data

electricty_price = data[0]["energy_price_DKK_per_kWh"]
price_import = data[0]["import_tariff_DKK/kWh"]
price_export = data[0]["export_tariff_DKK/kWh"]
#------------------------------------------
penalty_excess_import = data[0]["penalty_excess_import_DKK/kWh"]
penalty_excess_export = data[0]["penalty_excess_export_DKK/kWh"]
#-----------------------------------------------------
max_import = data[0]["max_import_kW"]
max_export = data[0]["max_export_kW"]

In [109]:
print(f"electricty_price: {electricty_price}")
print(f"price_import: {price_import}")
print(f"price_export: {price_export}")
print(f"penalty_excess_import: {penalty_excess_import}")
print(f"penalty_excess_export: {penalty_excess_export}")
print(f"max_import: {max_import}")
print(f"max_export: {max_export}")


electricty_price: [1.1, 1.05, 1.0, 0.9, 0.85, 1.01, 1.05, 1.2, 1.4, 1.6, 1.5, 1.1, 1.05, 1.0, 0.95, 1.0, 1.2, 1.5, 2.1, 2.5, 2.2, 1.8, 1.4, 1.2]
price_import: 0.5
price_export: 0.4
penalty_excess_import: 10
penalty_excess_export: 10
max_import: 1000
max_export: 500


In [110]:
"Ramping Up for Solar and FFL"
data_dir_appliance_params = data_dir / "appliance_params.json"
data_dir_appliance_params = data_dir_appliance_params.resolve()

with open(f"{data_dir_appliance_params}", "r") as f:
    data = json.load(f)

    data

max_power_solar = data["DER"][0]["max_power_kW"]
#min_power = data["DER"][0]["min_power_ratio"]
max_ramp_up_rate_solar = data["DER"][0]["max_ramp_rate_up_ratio"]
max_ramp_down_rate_solar = data["DER"][0]["max_ramp_rate_down_ratio"]
max_load_FFL = data["load"][0]["max_load_kWh_per_hour"]
min_load_FFL = data["load"][0]["min_load_ratio"]

max_ramp_up_rate_FFL= data["load"][0]["max_ramp_rate_up_ratio"]
max_ramp_down_rate_FFL= data["load"][0]["max_ramp_rate_down_ratio"]

In [111]:
print(f"max_power_solar: {max_power_solar}")
print(f"max_ramp_up_rate_solar: {max_ramp_up_rate_solar}")
print(f"max_ramp_down_rate_solar: {max_ramp_down_rate_solar}")
print(f"max_load_FFL: {max_load_FFL}")
print(f"min_load_FFL: {min_load_FFL}")
print(f"max_ramp_up_rate_FFL: {max_ramp_up_rate_FFL}")
print(f"max_ramp_down_rate_FFL: {max_ramp_down_rate_FFL}")


max_power_solar: 3.0
max_ramp_up_rate_solar: 1.0
max_ramp_down_rate_solar: 1.0
max_load_FFL: 3.0
min_load_FFL: 0.0
max_ramp_up_rate_FFL: 1.0
max_ramp_down_rate_FFL: 1.0


All units are in kW or KWh

In [112]:
t = range(len(Solar_CF))
t

range(0, 24)

In [113]:
"We set here Beta that will determine how conservative"
"will the model be with the deviation of the load of reference"
Beta = 0

In [114]:

# Create model
model = gp.Model("Prosumer_Optimization")

# Decision variables
x_imports = model.addVars(t, name="imported_power_kW", lb=0, ub=GRB.INFINITY)
x_exports = model.addVars(t, name="exported_power_kW", lb=0, ub=GRB.INFINITY)
x_FFL = model.addVars(t, name="FFL_consumption_kW", lb=0, ub=1)  # scaled 0-1
z_import_excess = model.addVars(t, name="excess_import_kW", lb=0, ub=GRB.INFINITY)
z_export_excess = model.addVars(t, name="excess_export_kW", lb=0, ub=GRB.INFINITY)
#Auxiliary value for the deviation in the objective function
d = model.addVars(t, lb=0, name="deviation")

# Hourly binary variables for tariffs
y_import = model.addVars(t, vtype=GRB.BINARY, name="tariff_import_activation")
y_export = model.addVars(t, vtype=GRB.BINARY, name="tariff_export_activation")

# Hourly constraints
for i in t:
    # FFL min/max
    model.addConstr(x_FFL[i] * max_load_FFL >= min_load_FFL, name=f"FFL_min_{i}")
    model.addConstr(x_FFL[i] * max_load_FFL <= max_load_FFL, name=f"FFL_max_{i}")

    # Energy balance: generation + imports - exports = load
    model.addConstr(
        x_imports[i] - x_exports[i] + Solar_CF[i] * max_power_solar == x_FFL[i] * max_load_FFL,
        name=f"energy_balance_{i}"
    )

    # Excess import/export
    model.addConstr(z_import_excess[i] >= x_imports[i] - max_import, name=f"excess_import_{i}")
    model.addConstr(z_export_excess[i] >= x_exports[i] - max_export, name=f"excess_export_{i}")

    # Binary tariff activation (big-M constraints)
    model.addConstr(x_imports[i] <= max_import * y_import[i], name=f"import_limit_{i}")
    model.addConstr(x_exports[i] <= max_export * y_export[i], name=f"export_limit_{i}")

    # Ramp constraints for FFL
    if i > 0:
        model.addConstr(
            (x_FFL[i] * max_load_FFL) - (x_FFL[i-1] * max_load_FFL) <= max_ramp_up_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_up_{i}"
        )
        model.addConstr(
            (x_FFL[i-1] * max_load_FFL) - (x_FFL[i] * max_load_FFL) <= max_ramp_down_rate_FFL * max_load_FFL,
            name=f"FFL_ramp_down_{i}"
        )
    #Auxiliary value for the deviation in the objective function
    model.addConstr(d[i] >= x_FFL[i] - FFL_ref[i], name=f"Deviation_above_0_{i}")
    model.addConstr(d[i] >= -(x_FFL[i] - FFL_ref[i]), name = f"Deviation_below_0_{i}")


# Objective: minimize cost + penalties + deviation from ref_load
model.setObjective(
    gp.quicksum(
        Beta * price_import * x_imports[i] + penalty_excess_import * z_import_excess[i] +
        price_export * x_exports[i] + penalty_excess_export * z_export_excess[i] +
        (1-Beta) * d[i] *max_load_FFL
        for i in t 
    ),
    sense=GRB.MINIMIZE
)

# Solve
model.optimize()


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G4 CPU @ 1.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 262 rows, 192 columns and 500 nonzeros
Model fingerprint: 0x2f58c6ec
Variable types: 144 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [4e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-02, 1e+03]
Found heuristic solution: objective 27.6210001
Presolve removed 262 rows and 192 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 1.344 27.621 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.344000000000e+00, best bound 1.344000000000e+00, gap 0.0000%


In [115]:

# Total FFL consumption over 24 hours
total_FFL = sum(x_FFL[i].X * max_load_FFL for i in t)
print("Total FFL consumption over 24h: {:.2f} kWh".format(total_FFL))

Total FFL consumption over 24h: 27.95 kWh


In [116]:
# Print results
print("Optimal objective value:", model.ObjVal)
for i in t:
    print(f"Hour {i}:")
    print(f"  Imported power = {x_imports[i].X:.2f} kW")
    print(f"  Exported power = {x_exports[i].X:.2f} kW")
    print(f"  FFL consumption = {x_FFL[i].X*max_load_FFL:.2f} kW")
    print(f"  FFL of reference = {FFL_ref[i]*max_load_FFL:.2f} kW")
    print(f"  Excess import = {z_import_excess[i].X:.2f} kW")
    print(f"  Excess export = {z_export_excess[i].X:.2f} kW")
    print(f"  Import tariff active = {int(y_import[i].X)}")
    print(f"  Export tariff active = {int(y_export[i].X)}")

Optimal objective value: 1.3439999999999999
Hour 0:
  Imported power = 0.17 kW
  Exported power = 0.00 kW
  FFL consumption = 0.17 kW
  FFL of reference = 0.17 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Export tariff active = 1
Hour 1:
  Imported power = 0.12 kW
  Exported power = 0.00 kW
  FFL consumption = 0.12 kW
  FFL of reference = 0.12 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Export tariff active = 1
Hour 2:
  Imported power = 0.12 kW
  Exported power = 0.00 kW
  FFL consumption = 0.12 kW
  FFL of reference = 0.12 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Export tariff active = 1
Hour 3:
  Imported power = 0.12 kW
  Exported power = 0.00 kW
  FFL consumption = 0.12 kW
  FFL of reference = 0.12 kW
  Excess import = 0.00 kW
  Excess export = 0.00 kW
  Import tariff active = 1
  Export tariff active = 1
Hour 4:
  Imported power = 0.22 kW
  Exported power = 0.

In [117]:
print("Optimal objective value:", model.ObjVal)
for i in t:
    print(f"Hour {i}:")
    print(f"  FFL consumption = {x_FFL[i].X*max_load_FFL:.2f} kW / {FFL_ref[i]*max_load_FFL:.2f} kW")
  

Optimal objective value: 1.3439999999999999
Hour 0:
  FFL consumption = 0.17 kW / 0.17 kW
Hour 1:
  FFL consumption = 0.12 kW / 0.12 kW
Hour 2:
  FFL consumption = 0.12 kW / 0.12 kW
Hour 3:
  FFL consumption = 0.12 kW / 0.12 kW
Hour 4:
  FFL consumption = 0.22 kW / 0.22 kW
Hour 5:
  FFL consumption = 1.44 kW / 1.44 kW
Hour 6:
  FFL consumption = 2.28 kW / 2.28 kW
Hour 7:
  FFL consumption = 2.40 kW / 2.40 kW
Hour 8:
  FFL consumption = 1.89 kW / 1.89 kW
Hour 9:
  FFL consumption = 0.66 kW / 0.66 kW
Hour 10:
  FFL consumption = 0.75 kW / 0.75 kW
Hour 11:
  FFL consumption = 1.05 kW / 1.05 kW
Hour 12:
  FFL consumption = 0.90 kW / 0.90 kW
Hour 13:
  FFL consumption = 0.84 kW / 0.84 kW
Hour 14:
  FFL consumption = 1.35 kW / 1.35 kW
Hour 15:
  FFL consumption = 1.95 kW / 1.95 kW
Hour 16:
  FFL consumption = 2.34 kW / 2.34 kW
Hour 17:
  FFL consumption = 2.70 kW / 2.70 kW
Hour 18:
  FFL consumption = 2.94 kW / 2.94 kW
Hour 19:
  FFL consumption = 2.64 kW / 2.64 kW
Hour 20:
  FFL consumption

## LP relaxation (to get duals):

In [118]:
# --- Solve LP relaxation (to get duals) ---
lp = model.copy()
for v in lp.getVars():
    if v.VType in (GRB.BINARY, GRB.INTEGER):
        v.VType = GRB.CONTINUOUS

lp.optimize()

if lp.Status == GRB.OPTIMAL:
    # Access duals directly by name
    for c in lp.getConstrs():
        print(f"{c.ConstrName}: Pi = {c.Pi}")

# Print results and duals
print("Optimal objective value:", model.ObjVal)


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-1035G4 CPU @ 1.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 262 rows, 192 columns and 500 nonzeros
Model fingerprint: 0x349d0df1
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [4e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-02, 1e+03]
Presolve removed 253 rows and 183 columns
Presolve time: 0.01s
Presolved: 9 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.836250e+00   0.000000e+00      0s
       7    1.3440000e+00   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.344000000e+00
FFL_min_0: Pi = 0.0
FFL_max_0: Pi = 0.0
energy_balance_0: Pi = 0.0
excess_import_0: Pi = 0.0
excess_expo

In [119]:
"Dual Variables"

print(lp.pi)
print(" ")
for constr in lp.getConstrs():
    print(f"{constr.ConstrName}: dual = {constr.Pi}, slack = {constr.Slack}")

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2000000000000002, 0.0, 0.0, 0.0, -0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2000000000000002, 0.0, 0.0, 0.0, -0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.